In [150]:
import os, fnmatch
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd

In [151]:
import sys
import json
import glob


In [152]:

BASE_PATH = '/data/archive/downstream/'
SOURCES = ['TARGET', 'TCGA', 'THR', 'TH']
def donor_to_samples(donor_id):
    sample_paths = glob.glob(BASE_PATH + donor_id + '*')
    sample_ids = [p[p.rfind('/')+1:] for p in sample_paths]
    return sample_ids

In [153]:
targetIds = donor_to_samples('TARGET')
thoIds = donor_to_samples('TH0')
thrIds = donor_to_samples('THR')
tgcaIds = donor_to_samples('TCGA')


In [154]:
listOfPathsToUMEND = sorted(glob.glob("/data/archive/downstream/" + 'TH0*/secondary/ucsc_cgl*/RSEM/Hugo' + '/rsem_genes.hugo.results'))
listOfPathsToDisease = sorted(glob.glob('/data/archive/downstream/TH0*/tertiary/treehouse-[8]*[!_compv5]/diagnosed_disease.txt'))

In [170]:
clinical = pd.read_csv("/data/archive/compendium/v5/clinical.tsv",sep='\t')

In [244]:
ribodDiagnosis = pd.read_csv("riboDepleted_samples_that_passedQC_and_have_known_diagnosis.tsv", sep='\t')

In [245]:
ribodDiagnosis=ribodDiagnosis.rename(columns={'Treehouse SAMPLE identifier':'th_sampleid','Diagnosis/Disease':'disease'})

In [246]:
methods = pd.read_csv("TranscriptMethod_THPEDv1.csv")

In [247]:
methods = methods.rename(columns={'Treehouse SAMPLE identifier':'th_sampleid'})

In [248]:
clinicalIdTissue = clinical[['th_sampleid','anat_sample','disease']] 

In [322]:
df = pd.merge(clinicalIdTissue, ribodDiagnosis, how='outer', on='th_sampleid')
df = pd.merge(df,methods,how='left',on='th_sampleid')
df.disease_y.fillna(df.disease_x, inplace=True)
df=df.rename(columns={'disease_y':'disease','anat_sample':'tissue'})

In [323]:
del df['disease_x']

In [326]:
df.tissue.fillna('NA',inplace=True)

In [330]:
ribodGlioblastomaSet = df.loc[df['TR_method']=='RiboMinus'].loc[df['disease']=='glioblastoma multiforme']

In [328]:
polyaGliomaBrainSet = df.loc[df['disease']=="glioma"].loc[df['tissue']=="brain"]

In [ ]:
compendium = pd.read_hdf("/data/archive/compendium/v5/v5_hugo_log2tpm.11340x58581.2018-02-03.hd5")

In [137]:
def index_containing_substring(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1

In [157]:

for filepath in listOfPathsToUMEND :
    currentSampleID = filepath[filepath.find('TH0'):filepath.find('TH0')+13]
    umendTable = pd.read_csv(filepath,sep='\t')
    if currentSampleID in listOfPathsToDisease:
        indexOfPath = index_containing_substring(listOfPathsToDisease,currentSampleID)
        f = open(listOfPathsToDisease[indexOfPath],'r')
        disease = f.read()
        f.close()
        umendTable['disease'] = np.repeat(disease,len(umendTable))
        totalUmendDisease = pd.concat([umendTable,totalUmendDisease])

KeyboardInterrupt: 